**NAME: SAIYYAM K. LODAYA<br>
  SAP ID: 60009220067<br>
  Batch: D2-2**

In [1]:
import pandas as pd

df_1 = pd.read_csv('/content/Breast_cancer_data.csv')
df_2 = pd.read_csv('/content/Social_Network_Ads.csv')
df_3 = pd.read_csv('/content/german_credit_data.csv')

____________________________ SCRATCH IMPLEMENTATION _____________________________________________


In [50]:
df_1.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [12]:
import math
import random
import pandas as pd
import numpy as np


In [13]:
def encode_class(mydata):
	classes = []
	for i in range(len(mydata)):
		if mydata[i][-1] not in classes:
			classes.append(mydata[i][-1])
	for i in range(len(classes)):
		for j in range(len(mydata)):
			if mydata[j][-1] == classes[i]:
				mydata[j][-1] = i
	return mydata


In [14]:
def splitting(mydata, ratio):
	train_num = int(len(mydata) * ratio)
	train = []
	test = list(mydata)
	while len(train) < train_num:
		index = random.randrange(len(test))
		train.append(test.pop(index))
	return train, test


In [15]:
def groupUnderClass(mydata):
	data_dict = {}
	for i in range(len(mydata)):
		if mydata[i][-1] not in data_dict:
			data_dict[mydata[i][-1]] = []
		data_dict[mydata[i][-1]].append(mydata[i])
	return data_dict


In [16]:
def MeanAndStdDev(numbers):
	avg = np.mean(numbers)
	stddev = np.std(numbers)
	return avg, stddev

def MeanAndStdDevForClass(mydata):
	info = {}
	data_dict = groupUnderClass(mydata)
	for classValue, instances in data_dict.items():
		info[classValue] = [MeanAndStdDev(attribute) for attribute in zip(*instances)]
	return info


In [17]:
def calculateGaussianProbability(x, mean, stdev):
	epsilon = 1e-10
	expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev + epsilon, 2))))
	return (1 / (math.sqrt(2 * math.pi) * (stdev + epsilon))) * expo

def calculateClassProbabilities(info, test):
	probabilities = {}
	for classValue, classSummaries in info.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, std_dev = classSummaries[i]
			x = test[i]
			probabilities[classValue] *= calculateGaussianProbability(x, mean, std_dev)
	return probabilities


In [18]:
def predict(info, test):
	probabilities = calculateClassProbabilities(info, test)
	bestLabel = max(probabilities, key=probabilities.get)
	return bestLabel

def getPredictions(info, test):
	predictions = [predict(info, instance) for instance in test]
	return predictions


In [19]:
def accuracy_rate(test, predictions):
	correct = sum(1 for i in range(len(test)) if test[i][-1] == predictions[i])
	return (correct / float(len(test))) * 100.0


In [26]:
# Load data using pandas
mydata = df_1.values.tolist()

# Encode classes and convert attributes to float
mydata = encode_class(mydata)
for i in range(len(mydata)):
	for j in range(len(mydata[i]) - 1):
		mydata[i][j] = float(mydata[i][j])


In [21]:
# Split the data into training and testing sets
ratio = 0.7
train_data, test_data = splitting(mydata, ratio)

print('Total number of examples:', len(mydata))
print('Training examples:', len(train_data))
print('Test examples:', len(test_data))


Total number of examples: 569
Training examples: 398
Test examples: 171


In [22]:
# Train the model
info = MeanAndStdDevForClass(train_data)

# Test the model
predictions = getPredictions(info, test_data)
accuracy = accuracy_rate(test_data, predictions)
print('Accuracy of the model:', accuracy)


Accuracy of the model: 100.0


In [35]:
from sklearn.metrics import f1_score

# Get true labels and predicted labels
true_labels = [example[-1] for example in test_data]
predicted_labels = predictions

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print F1 score
print('F1 score of the model:', f1)


F1 score of the model: 1.0


_________________________________________USING A MODEL________


In [42]:
df_2.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [43]:
df_3.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# Load data using pandas
df_2 = pd.read_csv('/content/Social_Network_Ads.csv')
df_3 = pd.read_csv('/content/german_credit_data.csv')

# One-hot encode categorical variables
df_2 = pd.get_dummies(df_2, columns=['Gender'], drop_first=True)  # Assuming 'Gender' is a categorical variable
df_3 = pd.get_dummies(df_3)  # One-hot encode all categorical variables

# Split data into training and testing sets
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_2.drop('Purchased', axis=1), df_2['Purchased'], test_size=0.2, random_state=0)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(df_3.drop('Risk_good', axis=1), df_3['Risk_good'], test_size=0.2, random_state=0)

# Create and train the Naive Bayes model for each dataset
gnb_2 = GaussianNB().fit(X_train_2, y_train_2)
gnb_3 = GaussianNB().fit(X_train_3, y_train_3)

# Make predictions
y_pred_2 = gnb_2.predict(X_test_2)
y_pred_3 = gnb_3.predict(X_test_3)

# Evaluate the model performance
accuracy_2 = accuracy_score(y_test_2, y_pred_2)
accuracy_3 = accuracy_score(y_test_3, y_pred_3)

print('Accuracy for Social_Network_Ads dataset:', accuracy_2)
print('Accuracy for german_credit_data dataset:', accuracy_3)

# Calculate F1 score for each dataset
f1_2 = f1_score(y_test_2, y_pred_2, average='weighted')
f1_3 = f1_score(y_test_3, y_pred_3, average='weighted')

# Print F1 scores
print('F1 score for Social_Network_Ads dataset:', f1_2)
print('F1 score for german_credit_data dataset:', f1_3)


Accuracy for Social_Network_Ads dataset: 0.925
Accuracy for german_credit_data dataset: 1.0
F1 score for Social_Network_Ads dataset: 0.9238498789346249
F1 score for german_credit_data dataset: 1.0
